In [1]:
%load_ext autoreload

In [2]:
import os
import time
from fastai.text import *

In [3]:
import sys
sys.path.append('./imdb_scripts/')


In [4]:
from create_toks import *
import sentencepiece as spm
from fastai.text import *

In [5]:
currvocab = 8
is_lowercase = True
curr_vocab_str = f"{currvocab}k"
if is_lowercase: 
    curr_vocab_str = curr_vocab_str + '_lowercase'

In [6]:
train_dir_str = 'data/aclImdb/train/all/'
all_name = 'train_all_lower.txt'

In [7]:
BOS = 'xbos'  # beginning-of-sentence tag
FLD = 'xfld'  # data field tag
SPM_MODEL_PATH=Path(f'data/aclImdb_spm/{curr_vocab_str}/')
PATH=Path('data/aclImdb/')

In [8]:
CLAS_PATH=Path('data/imdb_clas/')
LM_PATH=Path('data/imdb_lm/')
CLAS_PATH_SPM=Path('data/imdb_clas_spm/')
LM_PATH_SPM=Path('data/imdb_lm_spm/')

In [9]:
chunksize=24000

In [10]:
full_p = train_dir_str + all_name
to_txt = 'train_all_lower.txt'

In [11]:

arg_str = '--input='+ train_dir_str + to_txt + f' --model_prefix=model_' + curr_vocab_str + f' --vocab_size={currvocab}000'
print(arg_str)

--input=data/aclImdb/train/all/train_all_lower.txt --model_prefix=model_8k_lowercase --vocab_size=8000


In [12]:
spm.SentencePieceTrainer.Train(arg_str)

KeyboardInterrupt: 

# assuming lowering cas eis done in 64k file


In [13]:
def get_texts_spm(spm_model, df, n_lbls):
    tstart = time.time()
    if len(df.columns) == 1:
        labels = []
        texts = f'\n{BOS} {FLD} 1 ' + df[0].astype(str)
        texts = texts.apply(fixup).values.astype(str)
    else:
        labels = df.iloc[:,range(n_lbls)].values.astype(np.int64)
        texts = f'\n{BOS} {FLD} 1 ' + df[n_lbls].astype(str)
        for i in range(n_lbls+1, len(df.columns)): texts += f' {FLD} {i-n_lbls} ' + df[i].astype(str)
        texts = texts.apply(fixup).values.astype(str)

    #tok = proc_all_mp_smp(partition_by_cores(texts))
    tok = [spm_model.EncodeAsIds(t) for t in texts]
    tend = time.time()
    print(f'{(tend-tstart)/(len(texts)/1000):.2f}sec per 1k rows')
    return tok, list(labels)

In [14]:
def get_all_spm(spm_model, df, n_lbls):
    tok, labels = [], []
    for i, r in enumerate(df):
        
        print(i)
        tok_, labels_ = get_texts_spm(spm_model, r, n_lbls)
        tok += tok_;
        labels += labels_
        
    return tok, labels

In [15]:
sp8_lower = spm.SentencePieceProcessor()

In [16]:
sp8_lower.Load('data/imdb_lm_spm/model_8k_lowercase.model')

True

In [17]:
print(sp8_lower.EncodeAsPieces("I wish I knew what the FUCKingHELL is up with the thingy".lower()))

['▁i', '▁wish', '▁i', '▁knew', '▁what', '▁the', '▁fu', 'ck', 'ing', 'hell', '▁is', '▁up', '▁with', '▁the', '▁thing', 'y']


In [18]:
print(sp8_lower.EncodeAsPieces("Shittingduckcrappoopercrackingjack".lower()))

['▁shi', 't', 'ting', 'd', 'uck', 'c', 'ra', 'pp', 'oo', 'per', 'crack', 'ing', 'jack']


In [19]:
print(sp8_lower.EncodeAsPieces("reaaaaaaaaaaaaaaaaaally".lower()))

['▁re', 'a', 'aaaa', 'aaaa', 'aaaa', 'aaaa', 'ally']


In [20]:
df_trn = pd.read_csv(LM_PATH/'train_lower.csv', header=None, chunksize=chunksize)
df_val = pd.read_csv(LM_PATH/'test_lower.csv', header=None, chunksize=chunksize)

In [ ]:
#get_all_spm(sp32_lower, df_trn, 1)

In [21]:
tok_trn_spm, trn_labels_spm = get_all_spm(sp8_lower, df_trn, 1)
tok_val_spm, val_labels_spm = get_all_spm(sp8_lower, df_val, 1)

0
1.89sec per 1k rows
1
2.34sec per 1k rows
2
2.30sec per 1k rows
3
2.14sec per 1k rows
0
1.95sec per 1k rows


In [22]:
len(tok_trn_spm), len(tok_val_spm)

(90000, 10000)

In [23]:
(CLAS_PATH/'tmp').mkdir(exist_ok=True)

np.save(LM_PATH/'tmp'/'tok_trn_spm8_lower_ids.npy', tok_trn_spm)
np.save(LM_PATH/'tmp'/'tok_val_spm8_lower_ids.npy', tok_val_spm)


# now tokenize for classification

In [24]:
df_trn = pd.read_csv(CLAS_PATH/'train_lower.csv', header=None, chunksize=chunksize)
df_val = pd.read_csv(CLAS_PATH/'test_lower.csv', header=None, chunksize=chunksize)

In [25]:
tok_trn_spm, trn_labels_spm = get_all_spm(sp8_lower, df_trn, 1)
tok_val_spm, val_labels_spm = get_all_spm(sp8_lower, df_val, 1)

0
1.68sec per 1k rows
1
1.63sec per 1k rows
0
1.72sec per 1k rows
1
1.80sec per 1k rows


In [26]:
len(tok_trn_spm), len(tok_val_spm)

(25000, 25000)

In [27]:
np.save(CLAS_PATH/'tmp'/'tok_trn_spm8_lower_ids.npy', tok_trn_spm)
np.save(CLAS_PATH/'tmp'/'tok_val_spm8_lower_ids.npy', tok_val_spm)


In [28]:

# labels are no good. ignore. 
np.save(CLAS_PATH/'tmp'/'trn_labels_lower_spm8.npy', trn_labels_spm)
np.save(CLAS_PATH/'tmp'/'val_labels_lower_spm8.npy', val_labels_spm)